# Lending Club Case Study

**Version:** 1.0

---

**Document Information:**
- **Authors:** Irshad Shaikh and Ganesh Visweswaran
- **Creation Date:** May 8th, 2024
- **Last Update Date:** May 8th, 2024
- **Last Updated by:** [Update by Name/Initials]

---

# Importing Libraries

In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import warnings as ws
#import plotly.express as px
ws.filterwarnings('ignore')


print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)
print("Seaborn version:", sns.__version__)
print("Matplotlib version:", plt.matplotlib.__version__)


Pandas version: 1.3.4
Numpy version: 1.20.3
Seaborn version: 0.11.2
Matplotlib version: 3.4.3


# Data Sourcing
Our data source appears to be private, as it is associated with a specific organization.

In [ ]:
#----Loading the file into dataframe


loan_df=pd.read_csv('../Data_Source/loan.csv')
print ("Sample records: \n")
loan_df.head(2)

In [ ]:
#----Identify the number of columns and rows in dataset


print("Loan Data - Row vs columns: ", loan_df.shape)

# Data Cleaning
## 1.1 Clean all rows which are null
#### Created two functions 
1. detinfo(df,list) : It gives data type, Number of Null Values, Number of Unique Values and detailed information about the column 


In [ ]:
#function defined to provide detailed info on any column list when called for

def column_info(df, columns):
    """
    Print detailed information about specified columns in a DataFrame.

    Parameters:
    df : DataFrame
        The DataFrame containing the columns to analyze.
    columns : list
        List of column names for which information is needed.
    """

    for col in columns:
        print("*******************************************************")
        print("Column Name: ", col)
        print("-------------------------------------------------------")
        print("Type: ", df[col].dtype)
        print("Number of Null Values: ", df[col].isnull().sum())
        print("Number of Unique Values:", len(df[col].unique()))
        print("Detailed Information:")
        print(df[col].describe())

###  Identify the unnecessary columns

####  1 The columns which has 100% Null values and remove them

In [ ]:
100*loan_df.isnull().mean()[loan_df.isnull().mean()==1]

In [ ]:
empty_column=loan_df.isnull().mean()[100*loan_df.isnull().mean()==100].keys()
empty_column
loan_df_1=loan_df.drop(columns=empty_column)
loan_df_1.shape

In [ ]:
100*loan_df_1.isnull().mean().sort_values(ascending=False)

#### 2 We still have some columns which have more than 90% Null values hence removing them

In [ ]:
empty_column=loan_df_1.isnull().mean()[100*loan_df_1.isnull().mean()>90].keys()
empty_column
loan_df_1=loan_df_1.drop(columns=empty_column)
loan_df_1.shape

#### 3 Check for empty rows

In [ ]:
# There are no empty rows when we execute this
100*loan_df_1.isnull().mean(axis=1).sort_values(ascending=False)

#### 4 Remove duplicate records

In [ ]:
#There are no duplicate records
loan_df_1.shape , loan_df_1.drop_duplicates().shape

In [ ]:
# Viewing data to check info:
column_info(loan_df_1,loan_df_1.columns)

##### 5 Remove column wich doesn't have meaningfull data

In [ ]:
# After executing above we got the info that,
# we have some columns which has 3 or less than 3 unique values 
# Checking for for the column which has less than  3 unique values
limi_val=list()
for x in loan_df_1.columns:
    if len(loan_df_1[x].unique())<4:
        limi_val.append(x)
print(limi_val)

In [ ]:
# 
# Check for the values if they have any significance
for x in limi_val:
    print(x,'---->',loan_df_1[x].unique())

In [ ]:
# We don't need people who have loan_status='Current'
loan_df_1=loan_df_1[loan_df_1['loan_status']!='Current']
# These columns does not have anything important hence removing them
col=['pymnt_plan','initial_list_status','collections_12_mths_ex_med','policy_code','application_type'
     ,'acc_now_delinq','chargeoff_within_12_mths','delinq_amnt','tax_liens']
loan_df_1=loan_df_1.drop(columns=col)
loan_df_1.shape

In [ ]:
# checking random record to see if there are any columns which is not important for analysis
loan_df_1.iloc[1]

In [ ]:
# For now URL and DESC columns seems not so useful
loan_df_1=loan_df_1.drop(columns=['url','desc'])
loan_df_1.shape

##### 6 Remove special symbols in columns so that we can clean and segrigate data for analysis

In [ ]:
loan_df_1['int_rate']=loan_df_1['int_rate'].apply(lambda x: x.replace('%',''))
loan_df_1['zip_code']=loan_df_1['zip_code'].apply(lambda x: x.replace('xx',''))
loan_df_1['sub_grade']=loan_df_1['sub_grade'].apply(lambda x: x[-1])
loan_df_1['revol_util']=loan_df_1['revol_util'].apply(lambda x: np.nan if pd.isna(x) else x.replace('%',''))


##### 7 Data type corrections

In [ ]:
# Changing below columns to Numeric since it appear as string
loan_df_1['zip_code']=pd.to_numeric(loan_df_1['zip_code'])
loan_df_1['int_rate']=loan_df_1['int_rate'].str.strip().astype(float)
loan_df_1['revol_util']=loan_df_1['revol_util'].str.strip().astype(float)

In [ ]:
# Changing string columns to date 

loan_df_1['issue_d'] = pd.to_datetime(loan_df_1['issue_d'], format='%b-%y')
loan_df_1['last_pymnt_d'] = pd.to_datetime(loan_df_1['last_pymnt_d'], format='%b-%y')
loan_df_1['last_credit_pull_d'] = pd.to_datetime(loan_df_1['last_credit_pull_d'], format='%b-%y')

In [ ]:
#Identify numeric and string column
num_col=loan_df_1.dtypes[(loan_df_1.dtypes=='int64') | (loan_df_1.dtypes=='float64')]
str_col=loan_df_1.dtypes[~((loan_df_1.dtypes=='int64') | (loan_df_1.dtypes=='float64'))]
loan_df_1.columns

In [ ]:
# I have checked for the value of each column and seggrigated them as below 
ordered_cat=['term','grade','sub_grade','emp_length','issue_d','delinq_2yrs'
             ,'earliest_cr_line','inq_last_6mths','mths_since_last_delinq',
             'last_pymnt_d','last_credit_pull_d','pub_rec_bankruptcies']
unordered_cat=['home_ownership','verification_status','loan_status','purpose',
               'zip_code','addr_state']
other=['id','member_id','desc','emp_title','title']
measure_col=['loan_amnt', 'funded_amnt','funded_amnt_inv','int_rate',
             'installment','annual_inc','dti','open_acc','pub_rec','revol_bal'
             ,'revol_util','total_acc','out_prncp','total_pymnt'
             ,'total_pymnt_inv','total_rec_prncp','total_rec_int','total_rec_late_fee'
             ,'recoveries','collection_recovery_fee','last_pymnt_amnt']

In [ ]:
pd.set_option('display.float_format',lambda x: '%.3f' % x)
loan_df_1[measure_col].describe()


# Univariate Analysis




#### 1. Categorical variable Univariate Analysis

In [ ]:
# Plotting cateogorical variable and trying to see the pattern if we find any
for z in ordered_cat:
    print("--------------Ordered Category",z," -----------------")
    dta=loan_df_1[z].value_counts().head(15).reset_index()
    sns.barplot(y='index',x=z,data=dta)
    plt.xticks(rotation=90)
    plt.show()

According to above barplot 
* More people prefer to have 36 month tenuer
* More poople fall into B, A and C category
* 10+ years people tend to apply more for loan than anyother category
* There were huge rise in loan application in 2011 sep, oct, nov and dec 

In [ ]:
# Insights from unorder categorical variable
for z in unordered_cat:
    print("--------------unordered Category",z," -----------------")
    dta=loan_df_1[z].value_counts().head(15).reset_index()
    sns.barplot(y='index',x=z,data=dta)
    plt.xticks(rotation=90)
    plt.show()

According to above graphs:
 1. Home_owner who have MORTGAGE and RENT category tends to borrow as compare to any other category
 2. Not verified people are more in number who are getting loan approved this is serius alarm 
 3. People with debt_consolidation and credit_card tends to apply for loan more than any other purpose
 4. People who are from NY and CA tend to apply for loan more than any other state

In [ ]:
loan_df_1['loan_status'].value_counts(normalize=True)*100

#### 2. Numeric variable univariate analysis

In [ ]:
for var in measure_col:
    print("--------------Box Plot Category",z," -----------------")
    plt.figure(figsize=(10,6))
    print()
    sns.boxplot(data=loan_df_1,x=var)
    #plt.xticks(rotation=90)
    plt.show()
    
# 1. As per below there are some outliers in column collection_recovery_fee, recoveries, 
#    total_rec_late_fee,out_prncp, pub_rec, annual_inc

In [ ]:
# I am taking annual_inc to remove outlier from 
lis=loan_df_1['annual_inc'].tolist()
mn=np.mean(lis)
sdd=np.std(lis)
li=[x for x in lis if (x > mn - 2 * sdd)]
li=[x for x in li if (x < mn + 2 * sdd)]
loan_df_2=loan_df_1[loan_df_1['annual_inc'].isin(li)]

In [ ]:
# Comparing the previous and new dataframe
loan_df_1.shape , loan_df_2.shape

In [ ]:
# Plotting the same value
for var in measure_col:
    print("--------------Box Plot Category",z," -----------------")
    plt.figure(figsize=(10,6))
    sns.boxplot(data=loan_df_2,x=var)
    #plt.xticks(rotation=90)
    plt.show()

In [ ]:
loan_df_2.describe()

# Above box plot and below description of data after removing:
#  1. Most of the loan_amnt are between 5400 and 15000
#  2. Most of the int_rate are between 9.070 and 14.5


In [ ]:
# Plotting histplot for more insights
# The below give same insights which we can see above in numbers
# 1. Most of the loan_amnt and funded_amnt lies between 5400 and 15000
# 2. int_rate lies between 9% and 14.5%
# 3. Annual income of borrowers lies between 40000 and 80000

for z in measure_col:
    print("--------------Measure Category",z," -----------------")
    sns.histplot(loan_df_2[z])
    plt.show()

In [ ]:
# There are some numeric columns I belive if we can create categories
loan_df_2['cat_loan_amnt'] = pd.cut(x=loan_df_2['loan_amnt'], bins=[0, 10000, 20000, 30000, 40000], labels=['0-10K', '10K-20K', '20K-30K','30K-40K'])
loan_df_2['cat_annual_inc'] = pd.cut(x=loan_df_2['annual_inc'], bins=[0,20000, 40000, 60000, 80000,7000000], labels=['0-20K', '20K-40K', '40K-60K','60K-80K','>80K'])
loan_df_2['cat_int_rate']=pd.cut(x=loan_df_2['int_rate'],bins=[0,3,6,9,12,15,25],labels=['0-3%','3%-6%','6%-9%','9%-12%','12%-15%','>15%'])
##Create numerical column loan amount to annual income ratio
loan_df_2['lta']=loan_df_2['loan_amnt']/loan_df_2['annual_inc']
new_cat_list=['cat_loan_amnt','cat_annual_inc','cat_int_rate']

# Adding Newly derived Categorical columns to list
ordered_cat.extend(['cat_loan_amnt','cat_annual_inc','cat_int_rate'])
ordered_cat

In [ ]:
for i in measure_col:
    print("--------------loan status vs ",i," -----------------")
    sns.boxplot(x=loan_df_2['loan_status'],y=loan_df_2[i])
    plt.show()

In [ ]:
#numerical vs categorical
for i in measure_col:
    print("--------------loan status vs ",i," -----------------")
    sns.barplot(x=loan_df_2['loan_status'],y=loan_df_2[i])
    plt.show()

In [ ]:
# Plotting the graph
for z in new_cat_list:
    print("--------------New Category",z," -----------------")
    dta=loan_df_2[z].value_counts().head(15).reset_index()
    sns.barplot(y='index',x=z,data=dta)
    plt.xticks(rotation=90)
    plt.show()

# Segmentation Analysis

### 1. Creating segmentation on the basis of loan status and plotting the graph

In [ ]:
# Segmented on the basis of 'loan_status' and checking for each category
for z in ordered_cat:
    print("--------------Segmentation of Ordered Category",z," -----------------")
    sns.countplot(data=loan_df_2,hue='loan_status',x=z)
    plt.title(z.upper())
    plt.xticks(rotation=45)
    plt.show()

*  In Above chart we see that people who have fully paid as status they prefer to have 36 month as * * term
*  In GRADE most of the fully paid status falls into B, C and A grades while most of the defaulters * are in B, C and D grade
*  People who have EMP_LENGTH 10+ Years tends to apply more than any other category
   i. Most of the defaulter in EMP_LENGTH are 10+ years or <3 Years

In [ ]:
# Segmented on the basis of 'loan_status' and checking for unorder categorical variable
for z in unordered_cat:
    print("--------------Segmentation of unordered Category",z," -----------------")
    sns.countplot(data=loan_df_2,hue='loan_status',y=z,saturation=0.9,)
    plt.title(z.upper())
    plt.xticks(rotation=45)
    plt.show()

As per the above graph we get the below analysis:
* HOME_OWNERSHIP : Defaulters and Fully paid customers are either rented or have mortgage their property
* VERIFICATION_STATUS: Defaulters are almost same in numbers 
* More numbers of defaulter and fully paid people 

# Segmented Bivariate Analysis

In [ ]:
# I have checked for the value of each column and seggrigated them as below 
cat=['term','grade','emp_length','home_ownership','verification_status','purpose']
mes=['loan_amnt', 'funded_amnt','int_rate',
             'installment','annual_inc']
ct=1
for z in cat:
    for zx in mes:
        print("--------------Segmentation of Bivariate Analysis: ",z,'-',zx," -----------------")
        sns.heatmap(loan_df_2.pivot_table(columns=['loan_status'],index=[z],
                                  aggfunc='median',
                                  values=zx),
                    cmap='YlGnBu', annot=True, fmt=".2f")
        plt.title(f'Graph No.:{ct} Median Values of ({zx}) by Loan Status and {z}\n')
        plt.yticks(rotation=0)
        ct=ct+1
        plt.tight_layout()  
        plt.show()


# Insights in the above Graphs:
* Graph No. 1: Median loan amounts are almost the same between fully paid and charged-off customers.
* Graph No. 3: Median values of interest rates are higher for charged-off individuals who have applied for a 60-month term.
* Graph No. 4: Median installment values.
* Graph No. 6: For fully paid, Grade G has a greater median compared to any other category. Grade F tends to have a higher median loan amount in Charged off and Current status.
* Graph No. 8: Grades E, F, and G tend to accept loans at high interest rates.
* Graph No. 11: It seems that employees with more than 9 years of experience have a higher median compared to any other in charged off. Also, the current status shows that...
* Graph No. 13: Fully paid individuals have almost the same median throughout, the same goes for Charged off and Current.
* Graph No. 14: In charged off and fully paid loan status types, individuals with 9+ years of experience tend to have a higher number of median installments.
* Graph No. 15: Fully paid people's annual income tends to be higher than Charged off people's.
* Graph No. 16: Median values of loan amounts are high in the case of OTHER and MORTGAGE for charged off. Also, MORTGAGE individuals tend to have more loan amounts in Fully paid and Current categories.
* Graph No. 19: Median value of installments is higher in the case of home_ownership-Other in the charged-off category.
* Graph No. 26: Among loans categorized as "Charged Off" in loan status, the purposes of credit_card, debt_consolidation, and small business exhibit high median approved loan amounts.

# Multivariate Analysis

In [ ]:
mult_measure=['loan_amnt','installment','funded_amnt','funded_amnt_inv','int_rate'
             ,'annual_inc','dti','open_acc','revol_util','out_prncp','total_pymnt'
             ,'total_rec_prncp'
             ]
#Multivariate analysis
plt.figure(figsize=(10,8))
sns.heatmap(loan_df_2[mult_measure].corr(),annot=True)
plt.show()